In [1]:
# this code is to identify your ideal place to live based on your preferences

In [2]:
# Dependencies

import gmaps
import numpy as np
import pandas as pd
import requests
import time
import scipy.stats as st
from datetime import datetime
from scipy.stats import linregress
from matplotlib import pyplot as plt
from config import (gkey)

In [3]:
walkscore_map = pd.read_csv('Resources/walkscore_mapping.csv')
property_dataset = pd.read_csv('Resources/clean_property_value.csv')

In [4]:
bedrooms = input('How many bedrooms would you want?')
print(bedrooms)

How many bedrooms would you want?3
3


In [5]:
budget = input('What is your budget? (type digits only)')

What is your budget? (type digits only)300000


In [6]:
walk_score = input('Are you a walker? (Yes/No)')

Are you a walker? (Yes/No)Yes


In [7]:
bike_score = input('Are you a biker? (Yes/No)')

Are you a biker? (Yes/No)No


In [8]:
## rename region name to zip code
property_dataset = property_dataset.rename(columns={'RegionName':'ZipCode'})

## only use walkscore with zip code and scores (remove lat and lon)
walkscore_map = walkscore_map[['ZipCode','walk score','bike score']]
walkscore_map.head()

,ZipCode,walk score,bike score
0,94567,0,12.0
1,95554,0,NaN
2,96061,0,NaN
3,96113,0,17.0
4,95571,0,27.0


In [9]:
ideal_place = pd.merge(property_dataset,walkscore_map,on='ZipCode')
ideal_place = ideal_place.sort_values(by='CAGR', ascending=False)
ideal_place.head()

,ZipCode,RegionType,StateName,State,City,Metro,CountyName,1/31/14,2/28/21,3/31/21,Bdrm,CAGR,Lat,Lng,walk score,bike score
3451,95822,Zip,CA,CA,Sacramento,Sacramento--Roseville--Arden-Arcade,Sacramento County,115210.0,390018,398346,1,245.756445,38.512871,-121.497029,87,64.0
3090,90211,Zip,CA,CA,Beverly Hills,Los Angeles-Long Beach-Anaheim,Los Angeles County,789562.0,2575394,2551752,3,223.185766,34.066107,-118.384178,56,45.0
3612,95828,Zip,CA,CA,Florin,Sacramento--Roseville--Arden-Arcade,Sacramento County,94743.0,293825,301028,1,217.731125,38.492988,-121.388467,83,62.0
3905,90266,Zip,CA,CA,Manhattan Beach,Los Angeles-Long Beach-Anaheim,Los Angeles County,804901.0,2540611,2511376,1,212.010545,33.890837,-118.398784,73,95.0
3858,95816,Zip,CA,CA,Sacramento,Sacramento--Roseville--Arden-Arcade,Sacramento County,179658.0,547554,551084,1,206.740585,38.580018,-121.462758,19,8.0


In [10]:
ideal_place_bdr = ideal_place.loc[ideal_place['Bdrm']==int(bedrooms)]
ideal_place_budget = ideal_place_bdr.loc[ideal_place_bdr['3/31/21']<=int(budget)]

In [11]:
if walk_score == 'Yes':
    ideal_place = ideal_place_budget.sort_values(by=['CAGR','walk score', '3/31/21'], ascending=False)
    ideal_place = ideal_place[['ZipCode','City','CountyName','3/31/21','CAGR','walk score','bike score']]

In [12]:
print(f'''
Based on your preferences in bedrooms({bedrooms}), budget (${budget}), walking preferences({walk_score}) and biking preferences ({bike_score}), your ideal top 5 cities to live, with the highest CAGR, are:''')
ideal_place.head()


Based on your preferences in bedrooms(3), budget ($300000), walking preferences(Yes) and biking preferences (No), your ideal top 5 cities to live, with the highest CAGR, are:


,ZipCode,City,CountyName,3/31/21,CAGR,walk score,bike score
3876,95205,Stockton,San Joaquin County,287941,148.754676,38,61.0
3889,93505,California City,Kern County,193097,125.238540,4,26.0
2440,95388,Winton,Merced County,270790,123.107471,50,44.0
3883,95968,Palermo,Butte County,269714,115.495366,82,86.0
2644,93721,Fresno,Fresno County,258827,115.477281,17,29.0
